# Setup

In [ ]:
from tqdm import tqdm
import numpy as np
import pandas as pd
import time
import datetime
import sys
import matplotlib.pyplot as plt
import seaborn as sns
import os
import shutil
import json
import pickle


np.random.seed(42)

from formats import experiment_pb2
from formats import  quantification_pb2

from skimage import io
import pandas as pd
import utils


from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler

import torch
from torch.utils.data import Dataset, DataLoader, ConcatDataset, ChainDataset


import os
import pyro
import pyro.distributions as dist
import pyro.poutine
from pyro.infer import MCMC, NUTS
import math
import torch.nn as nn
import torch.nn.functional as F
from pyro.infer import SVI, Trace_ELBO
from pyro.optim import Adam
from pyro.infer.autoguide.guides import AutoDiagonalNormal
import pyro.distributions.constraints as constraints
from tqdm import trange

import utils



In [ ]:
local_radius_px = 5

In [ ]:
import data
dataset = data.get_dataset(local_radius_px=local_radius_px)

In [ ]:
x,q = next(iter(torch.utils.data.DataLoader(dataset=dataset,batch_size=len(dataset))))

# from sklearn.preprocessing import StandardScaler
# x_shape = x.shape
# x = StandardScaler().fit_transform(x.reshape(-1,x.shape[-1])).reshape(x_shape)
# q = StandardScaler().fit_transform(q)

# x = torch.Tensor(x)
# q = torch.Tensor(q)

x /= x.shape[1]*x.shape[2]/q.shape[1]


In [ ]:
def eval_reconstruction(reconstruction):
    x_hat, q_hat = reconstruction
    
    x_flat = x.reshape(-1,x.shape[-1])
    x_hat = x_hat.reshape(-1,x.shape[-1])

    x_mean = x_flat.mean(0)
    x_ssr = (x_flat - x_hat).pow(2).sum()
    x_sst = (x_flat - x_mean).pow(2).sum()
    x_r2 = 1 - x_ssr/x_sst

    q_mean = q.mean(0)
    q_ssr = (q - q_hat).pow(2).sum()
    q_sst = (q - q_mean).pow(2).sum()
    q_r2 = 1 - q_ssr/q_sst

    return x_r2.item(), q_r2.item()


In [ ]:
def joint_pca_reconstruct(n_components):
    combined_data = torch.cat((q,x.reshape((x.shape[0],-1))),1)
    pca = PCA(n_components=n_components)
    pca.fit(combined_data)
    recon = torch.tensor(pca.inverse_transform(pca.transform(combined_data)))

    q_hat = recon[:,:q.shape[1]] 
    x_hat = recon[:,q.shape[1]:].reshape(x.shape) 
    
    return x_hat, q_hat

def joint_pca_latents(n_components):
    combined_data = torch.cat((q,x.reshape((x.shape[0],-1))),1)
    pca = PCA(n_components=n_components)
    return pca.fit_transform(combined_data)

In [ ]:
eval_reconstruction(
    joint_pca_reconstruct(n_components=1)
)

In [ ]:
eval_reconstruction(
    joint_pca_reconstruct(n_components=2)
)

In [ ]:
eval_reconstruction(
    joint_pca_reconstruct(n_components=3)
)